In [20]:
import pandas as pd
from itertools import product

def parse_table(tables, parents):
    parsed_tables = {}
    for key, vars in parents.items():
        if vars is not None:
            vars_list = list(vars)
            parents_num = len(vars)
            combos = product([True, False], repeat=parents_num)
            combos_list = list(combos)
            prob_list = list(tables[key])
            df_list = list()
            i = 4*parents_num #num of prob values in output of A
            j = 0
            while j != i:
                df_list.append(prob_list[j])
                j += 2

            df_key=pd.DataFrame(df_list, columns=[key])
            cpt = pd.DataFrame(combos_list, columns=vars_list)
            cpt[key] = df_key
            parsed_tables[key] = cpt

        else:
            combos_list = [True, False]
            cpt = pd.DataFrame(combos_list, columns=[key])
            prob_list = list(tables[key])
            df_key=pd.DataFrame(prob_list)
            cpt['withself'] = df_key
        parsed_tables[key] = cpt
    return parsed_tables

parents = {'B': None, 'E': None, 'A': ['B', 'E'], 'J': ['A'], 'M': ['A']}
tables = {'B': [0.001, 0.999], 'E': [0.002, 0.998], 'A': [0.95, 0.05, 0.94, 0.06, 0.29, 0.71, 0.001, 0.999], 'J': [0.9, 0.1, 0.05, 0.95], 'M': [0.7, 0.3, 0.01, 0.99]}

parsed_tables = parse_table(tables, parents)
for key, table in parsed_tables.items():
    print("Table for variable", key)
    print(table)

Table for variable B
       B  withself
0   True     0.001
1  False     0.999
Table for variable E
       E  withself
0   True     0.002
1  False     0.998
Table for variable A
       B      E      A
0   True   True  0.950
1   True  False  0.940
2  False   True  0.290
3  False  False  0.001
Table for variable J
       A     J
0   True  0.90
1  False  0.05
Table for variable M
       A     M
0   True  0.70
1  False  0.01
